In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
        
    return vocab


## TWEETSENT

In [6]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [7]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [8]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [9]:
text_Tweetsent = df_tweet['text']
polarity_ = np.asarray(df_tweet['sentiment'])

In [10]:
X_text = count_vect.transform(text_Tweetsent)

X_test_TWEET = tfidf_transformer.transform(X_text) # Aplicando o TF-IDF
X_test_TWEET.shape

(11533, 17314)

## TASH

In [11]:
df1 = pd.read_csv("tash-pt.csv")
df1 = df1.dropna()
df1 = df1.drop(columns=['id_twitter'])


In [12]:
df1['text'] = df1['text'].apply(remove_user)
df1['text'] = df1['text'].apply(Tokenize)

In [13]:
text_TASH = df1['text']
sentiment_ = np.asarray(df1['sentiment'])


X_text_TASH = count_vect.transform(text_TASH)

X_test_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_test_TASH_.shape

(2787, 17314)

## KANSOAN

In [3]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [4]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [5]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

X_train_KANSOAN = count_vect.fit_transform(text_KANSOAN)
X_train_KANSOAN = tfidf_transformer.fit_transform(X_train_KANSOAN) # Aplicando o TF-IDF
X_train_KANSOAN.shape

(9224, 17314)

## UNILEX

In [14]:
df1 = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [15]:
df_all = df_all.dropna()

In [16]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [17]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [18]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [19]:
Tweet = count_vect.transform(Tweet)
X_test_UNILEX = tfidf_transformer.transform(Tweet) # Aplicando o TF-IDF
X_test_UNILEX.shape

(12626, 17314)

In [20]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

sentiment = df['sentiment'].apply(func)

## NB - TRAIN -> KANSOAN- TEST -> TWEET

In [21]:
## TIRAR OS SPLITS , até no treino

In [22]:
clf = MultinomialNB()
clf.fit(X_train_KANSOAN, sentiment)
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred) )

              precision    recall  f1-score   support

          -1       0.36      0.90      0.51      3388
           0       0.00      0.00      0.00      2928
           1       0.67      0.38      0.49      5217

    accuracy                           0.44     11533
   macro avg       0.34      0.43      0.33     11533
weighted avg       0.41      0.44      0.37     11533



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NB - TRAIN -> KANSOAN - TEST -> UNILEX

In [23]:
clf = MultinomialNB()
clf.fit(X_train_KANSOAN, sentiment)
y_pred = clf.predict(X_test_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.40      0.87      0.55      4186
         0.0       0.00      0.00      0.00      4740
         1.0       0.36      0.36      0.36      3700

    accuracy                           0.39     12626
   macro avg       0.26      0.41      0.30     12626
weighted avg       0.24      0.39      0.29     12626



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NB - TRAIN -> KANSOAN - TEST -> TASH

In [24]:
clf = MultinomialNB()
clf.fit(X_train_KANSOAN, sentiment)
y_pred = clf.predict(X_test_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.37      0.84      0.52       881
           0       0.00      0.00      0.00      1018
           1       0.41      0.36      0.39       888

    accuracy                           0.38      2787
   macro avg       0.26      0.40      0.30      2787
weighted avg       0.25      0.38      0.29      2787



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## SVM - TRAIN->KANSOAN - TEST-> TWEET

In [25]:
clf = SVC().fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.38      0.80      0.51      3388
           0       0.00      0.00      0.00      2928
           1       0.62      0.52      0.57      5217

    accuracy                           0.47     11533
   macro avg       0.33      0.44      0.36     11533
weighted avg       0.39      0.47      0.41     11533



## SVM T KANSOAN T UNILEX

In [26]:
clf = SVC().fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.41      0.66      0.50      4186
         0.0       0.00      0.00      0.00      4740
         1.0       0.33      0.53      0.41      3700

    accuracy                           0.37     12626
   macro avg       0.25      0.39      0.30     12626
weighted avg       0.23      0.37      0.29     12626



## SVM T KANSOAN T TASH

In [27]:
clf = SVC().fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_TASH_)

print(classification_report(sentiment_,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.37      0.69      0.48       881
           0       1.00      0.00      0.00      1018
           1       0.39      0.51      0.44       888

    accuracy                           0.38      2787
   macro avg       0.59      0.40      0.31      2787
weighted avg       0.61      0.38      0.29      2787



## RL T KANSOAN T TWEET

In [28]:
clf = LogisticRegression(max_iter=1000).fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.39      0.77      0.52      3388
           0       0.00      0.00      0.00      2928
           1       0.61      0.57      0.59      5217

    accuracy                           0.48     11533
   macro avg       0.33      0.45      0.37     11533
weighted avg       0.39      0.48      0.42     11533



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RL T KANSOAN T UNILEX

In [29]:
clf = LogisticRegression(max_iter=1000).fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.44      0.58      0.50      4186
         0.0       0.00      0.00      0.00      4740
         1.0       0.31      0.60      0.41      3700

    accuracy                           0.37     12626
   macro avg       0.25      0.39      0.30     12626
weighted avg       0.24      0.37      0.29     12626



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RL T KANSOAN T TASH


In [30]:
clf = LogisticRegression(max_iter=1000).fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.38      0.69      0.49       881
           0       0.00      0.00      0.00      1018
           1       0.39      0.51      0.44       888

    accuracy                           0.38      2787
   macro avg       0.25      0.40      0.31      2787
weighted avg       0.24      0.38      0.29      2787



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
one = OneHotEncoder(sparse=False)
teste_ = one.fit_transform(np.asarray(sentiment).reshape(-1,1))
nome = pd.DataFrame(teste_)
nome['coluna'] = 0
nome.rename(columns={1: 2, 'coluna':1} , inplace=True)
Y_KANSOAN = nome[[0,1,2]].to_numpy()

In [36]:
def func(num):
    if num == -1:
        return 0
    else:
        return 1

sentiment = df['sentiment'].apply(func)

## TOTALMENT.C T KANSOAN T TWEET

In [37]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(1 , activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

y_one = np.asarray(sentiment)
y_one_test = one.fit_transform(polarity_.reshape(-1,1))

fit = model.fit(X_train_KANSOAN.todense(), y_one, epochs=5)

Epoch 1/5
289/289 [==============================] - 1s 5ms/step - loss: 0.1120 - accuracy: 0.9975
Epoch 2/5
289/289 [==============================] - 1s 5ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 3/5
289/289 [==============================] - 1s 5ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 4/5
289/289 [==============================] - 2s 5ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 5/5
289/289 [==============================] - 1s 5ms/step - loss: 0.0021 - accuracy: 1.0000


In [38]:
predicted = model.predict(X_test_TWEET.todense())

print(classification_report(np.argmax(one.fit_transform(polarity_.reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.29      1.00      0.45      3388
           1       0.00      0.00      0.00      2928
           2       0.00      0.00      0.00      5217

    accuracy                           0.29     11533
   macro avg       0.10      0.33      0.15     11533
weighted avg       0.09      0.29      0.13     11533



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## UNILEX

In [39]:
predicted = model.predict(X_test_UNILEX.todense())

print(classification_report(np.argmax(one.fit_transform(polarity.reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50      4186
           1       0.00      0.00      0.00      4740
           2       0.00      0.00      0.00      3700

    accuracy                           0.33     12626
   macro avg       0.11      0.33      0.17     12626
weighted avg       0.11      0.33      0.17     12626



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TASH

In [43]:
predicted = model.predict(X_test_TASH_.todense())

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1), zero_division=True))

              precision    recall  f1-score   support

           0       0.32      1.00      0.48       881
           1       1.00      0.00      0.00      1018
           2       1.00      0.00      0.00       888

    accuracy                           0.32      2787
   macro avg       0.77      0.33      0.16      2787
weighted avg       0.78      0.32      0.15      2787



## LSTM T KANSOAN T UNILEX

In [44]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(text_KANSOAN )))

vocab = set_array(text_KANSOAN )

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


15000

In [51]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

y_one = np.asarray(sentiment)


fit = model.fit(np.asarray(pre_X(text_KANSOAN)), y_one, epochs=5, batch_size=128)

Epoch 1/5
73/73 [==============================] - 9s 118ms/step - loss: 0.2826 - accuracy: 0.9959
Epoch 2/5
73/73 [==============================] - 9s 121ms/step - loss: 0.0268 - accuracy: 1.0000
Epoch 3/5
73/73 [==============================] - 9s 118ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 4/5
73/73 [==============================] - 9s 124ms/step - loss: 0.0108 - accuracy: 1.0000
Epoch 5/5
73/73 [==============================] - 9s 124ms/step - loss: 0.0083 - accuracy: 1.0000


In [52]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50      4186
           1       0.00      0.00      0.00      4740
           2       0.00      0.00      0.00      3700

    accuracy                           0.33     12626
   macro avg       0.11      0.33      0.17     12626
weighted avg       0.11      0.33      0.17     12626



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TWEET

In [53]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.29      1.00      0.45      3388
           1       0.00      0.00      0.00      2928
           2       0.00      0.00      0.00      5217

    accuracy                           0.29     11533
   macro avg       0.10      0.33      0.15     11533
weighted avg       0.09      0.29      0.13     11533



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TASH

In [54]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.32      1.00      0.48       881
           1       0.00      0.00      0.00      1018
           2       0.00      0.00      0.00       888

    accuracy                           0.32      2787
   macro avg       0.11      0.33      0.16      2787
weighted avg       0.10      0.32      0.15      2787



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CONV1D T KANSOAN T TASH

In [56]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

y_one = np.asarray(sentiment)


fit = model.fit(np.asarray(pre_X(text_KANSOAN)), y_one, epochs=5, batch_size=128)

Epoch 1/5
73/73 [==============================] - 2s 23ms/step - loss: 0.0639 - accuracy: 1.0000
Epoch 2/5
73/73 [==============================] - 2s 23ms/step - loss: 2.4881e-07 - accuracy: 1.0000
Epoch 3/5
73/73 [==============================] - 2s 23ms/step - loss: 2.3968e-07 - accuracy: 1.0000
Epoch 4/5
73/73 [==============================] - 2s 23ms/step - loss: 2.3061e-07 - accuracy: 1.0000
Epoch 5/5
73/73 [==============================] - 2s 23ms/step - loss: 2.1960e-07 - accuracy: 1.0000


In [57]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.32      1.00      0.48       881
           1       0.00      0.00      0.00      1018
           2       0.00      0.00      0.00       888

    accuracy                           0.32      2787
   macro avg       0.11      0.33      0.16      2787
weighted avg       0.10      0.32      0.15      2787



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TWEET

In [58]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.29      1.00      0.45      3388
           1       0.00      0.00      0.00      2928
           2       0.00      0.00      0.00      5217

    accuracy                           0.29     11533
   macro avg       0.10      0.33      0.15     11533
weighted avg       0.09      0.29      0.13     11533



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## UNILEX

In [59]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50      4186
           1       0.00      0.00      0.00      4740
           2       0.00      0.00      0.00      3700

    accuracy                           0.33     12626
   macro avg       0.11      0.33      0.17     12626
weighted avg       0.11      0.33      0.17     12626



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## BDR T KANSOAN T TWEET

In [67]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=1,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1)),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

y_one = np.asarray(sentiment)

fit = model.fit(np.asarray(pre_X(text_KANSOAN)), y_one, epochs=5, batch_size=128)

Epoch 1/5
73/73 [==============================] - 2s 25ms/step - loss: 0.6087 - accuracy: 0.9836
Epoch 2/5
73/73 [==============================] - 2s 26ms/step - loss: 0.4226 - accuracy: 1.0000
Epoch 3/5
73/73 [==============================] - 2s 24ms/step - loss: 0.2849 - accuracy: 1.0000
Epoch 4/5
73/73 [==============================] - 2s 30ms/step - loss: 0.2046 - accuracy: 1.0000
Epoch 5/5
73/73 [==============================] - 2s 25ms/step - loss: 0.1555 - accuracy: 1.0000


In [68]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.29      1.00      0.45      3388
           1       0.00      0.00      0.00      2928
           2       0.00      0.00      0.00      5217

    accuracy                           0.29     11533
   macro avg       0.10      0.33      0.15     11533
weighted avg       0.09      0.29      0.13     11533



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## UNILEX

In [69]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50      4186
           1       0.00      0.00      0.00      4740
           2       0.00      0.00      0.00      3700

    accuracy                           0.33     12626
   macro avg       0.11      0.33      0.17     12626
weighted avg       0.11      0.33      0.17     12626



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## TASH

In [70]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.32      1.00      0.48       881
           1       0.00      0.00      0.00      1018
           2       0.00      0.00      0.00       888

    accuracy                           0.32      2787
   macro avg       0.11      0.33      0.16      2787
weighted avg       0.10      0.32      0.15      2787



c:\users\ddayv\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
